In [53]:
import yaml
import os

In [54]:
# Data wrangling
import pandas as pd
import numpy as np

In [55]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm import tqdm

In [56]:
# Models
from sklearn.ensemble import RandomForestRegressor

In [57]:
#Hyperparameter optimization
import optuna

In [58]:
# Models Pipelines
from sklearn.pipeline import Pipeline

In [59]:
# Model evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [60]:
import mlflow
import mlflow.sklearn

In [61]:
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("distillation_tower")

<Experiment: artifact_location=('file:///c:/Users/mjkipsz2/OneDrive - The University of '
 'Manchester/Desktop/distillation_colum_yield/notebooks/mlruns/550582570200691247'), creation_time=1736352237514, experiment_id='550582570200691247', last_update_time=1736352237514, lifecycle_stage='active', name='distillation_tower', tags={}>

In [62]:
# Save model
import joblib

In [63]:
# Load the configuration file
with open('../config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)

# Access the settings
project_folder = config['projectFolder']
df_path = os.path.join(project_folder, config['transformedDataFile'])
df = pd.read_csv(df_path)

# Display the first few rows
df.head()

,PressureC1_diff,FlowC1,Temp1,Yield
0,0.0000,432.0636,139.9857,69.400623
1,-9.9628,487.4029,131.0470,66.532666
2,-0.0695,437.3516,118.2666,71.102193
3,0.2257,481.8314,118.1769,69.793481
4,-0.1678,412.6471,120.7891,71.489516


In [64]:
# Define features (X) and target variable (y)
X = df.drop('Yield', axis=1)  # Features (all columns except 'Job Offer')
y = df['Yield']  # Target variable

In [65]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Check the shape of the data
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (202, 3)
Testing data shape: (51, 3)


In [66]:
def get_metrics(y_true, y_pred):

    dict_metrics = {
        'MSE': mean_squared_error(y_true, y_pred),
        'MAE': mean_absolute_error(y_true, y_pred),
        'R2 Score': r2_score(y_true, y_pred),
        
    }
    return dict_metrics

In [67]:
# Creating pipeline with Random Forest
pip_model_rf = Pipeline(steps=[
    ('model', RandomForestRegressor(random_state=2023))
])

# Fit pipeline with PCA
pip_model_rf.fit(X_train, y_train)

# Generate Predictions using the correctly fitted pipeline
y_pred = pip_model_rf.predict(X_test)

# Evaluate Metrics
metrics = get_metrics(y_test, y_pred)

# View Results
metrics

{'MSE': 0.5588840302677203,
 'MAE': 0.5822903208509888,
 'R2 Score': 0.7260009601685249}

In [68]:
#Hyperparameters optimization with Optuna

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    # Create a pipeline with the suggested hyperparameters
    pip_model_rf = Pipeline(steps=[
        ('model', RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=2023
        ))
    ])

    # Fit the pipeline
    pip_model_rf.fit(X_train, y_train)

    # Predict on the validation set
    y_pred = pip_model_rf.predict(X_test)

    # Calculate the mean squared error
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Create an Optuna study
study = optuna.create_study(direction='minimize')  # Minimize MSE
study.optimize(objective, n_trials=50)  # Run 50 trials

# Get the best hyperparameters
best_params = study.best_params
print("Best Parameters:", best_params)


[I 2025-01-09 15:39:38,078] A new study created in memory with name: no-name-44866442-5a36-49ed-9d8e-db6d5538d5f8
[I 2025-01-09 15:39:38,410] Trial 0 finished with value: 0.5697561784240338 and parameters: {'n_estimators': 274, 'max_depth': 46, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.5697561784240338.
[I 2025-01-09 15:39:38,482] Trial 1 finished with value: 1.0339824184255388 and parameters: {'n_estimators': 66, 'max_depth': 24, 'min_samples_split': 20, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.5697561784240338.
[I 2025-01-09 15:39:38,755] Trial 2 finished with value: 1.0133399101852134 and parameters: {'n_estimators': 227, 'max_depth': 47, 'min_samples_split': 14, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.5697561784240338.
[I 2025-01-09 15:39:38,951] Trial 3 finished with value: 0.571796950219291 and parameters: {'n_estimators': 187, 'max_depth': 44, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 0 with val

Best Parameters: {'n_estimators': 226, 'max_depth': 38, 'min_samples_split': 2, 'min_samples_leaf': 1}


In [69]:
# Start an MLflow run with a specific run name
with mlflow.start_run(run_name="rf_regression_run"):
    # Train the final model
    final_model = RandomForestRegressor(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        random_state=2023
    )
    final_model.fit(X_train, y_train)

    # Add model signature and input example
    signature = mlflow.models.infer_signature(X_train, y_train)
    input_example = X_train.iloc[:5]

    # Log metrics
    y_pred = final_model.predict(X_test)
    metrics = get_metrics(y_test, y_pred)
    mlflow.log_metrics(metrics)

    # Log parameters
    mlflow.log_params(best_params)

    # Log the model with signature and input example
    mlflow.sklearn.log_model(
        final_model, 
        "random_forest_model",
        signature=signature,
        input_example=input_example
    )

In [70]:
# Visualize the optimization history
optuna.visualization.plot_optimization_history(study).show()

In [71]:
# Visualize the parameter importance
optuna.visualization.plot_param_importances(study).show()

In [72]:
#joblib.dump(tuned_model, 'CIC_hybrid_no_outliers_Norsok_scikit.joblib')